In [5]:
import GMCM_class as G
import pandas as pd

## GMCM

In [15]:
data = pd.read_csv("travel insurance.csv")

In [16]:
data

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,81
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,71
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65,AUSTRALIA,-49.5,29.70,NaN,32
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60,AUSTRALIA,-39.6,23.76,NaN,32
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79,ITALY,-19.8,11.88,NaN,41
...,...,...,...,...,...,...,...,...,...,...,...
63321,JZI,Airlines,Online,Basic Plan,No,111,JAPAN,35.0,12.25,M,31
63322,JZI,Airlines,Online,Basic Plan,No,58,CHINA,40.0,14.00,F,40
63323,JZI,Airlines,Online,Basic Plan,No,2,MALAYSIA,18.0,6.30,M,57
63324,JZI,Airlines,Online,Basic Plan,No,3,VIET NAM,18.0,6.30,M,63


In [17]:
G = G.GaussianCopulaSynthesizer(data)

In [18]:
G._identify_columns()

In [19]:
G.convert_datetime_to_numerical()

In [20]:
G.handle_missing_values()

In [21]:
G.assign_intervals()

In [22]:
G.preprocess_data()

100%|██████████| 7/7 [02:48<00:00, 24.00s/it]


In [23]:
G.fit_distributions()

In [24]:
G.compute_gmm_cdf()

In [25]:
G.standard_gaussian_all()

In [26]:
G.generate_synthetic_data(63326)

100%|██████████| 63326/63326 [1:40:32<00:00, 10.50it/s]


,Distribution Channel,Agency,Gender,Age,Destination,Product Name,Agency Type,Claim,Duration,Net Sales,Commision (in value)
0,0.653848,0.333887,0.337998,45.617321,0.789538,0.536488,0.485246,0.704722,94.778994,98.565810,9.824353
1,0.568503,0.749622,0.329845,48.630437,0.931347,0.210102,0.879582,0.380870,64.811107,34.955860,0.000839
2,0.112954,0.233323,0.666745,35.997935,0.255475,0.394081,0.413971,0.535486,110.837463,29.442025,0.000000
3,0.468958,0.768560,0.214780,36.001763,0.683743,0.804757,0.861855,0.440527,112.594316,53.093205,20.069770
4,0.627169,0.883832,0.151215,35.124726,0.246415,0.969928,0.395553,0.308515,7.990980,321.260616,41.578165
...,...,...,...,...,...,...,...,...,...,...,...
63321,0.546730,0.368518,0.885219,21.307471,0.416798,0.598923,0.297873,0.816149,361.510075,17.289209,0.000575
63322,0.492971,0.444822,0.855452,28.537534,0.713901,0.126957,0.465274,0.384644,6.248301,11.025025,0.000000
63323,0.334145,0.200646,0.141117,58.101103,0.447194,0.360729,0.857299,0.480840,15.803596,19.777490,0.000908
63324,0.526884,0.240954,0.186885,54.186845,0.114471,0.431103,0.216395,0.467027,1.217148,7.760605,0.000000


In [27]:
df = G.post_process()

In [28]:
G.get_execution_times_df()

,Process,Time (seconds)
0,Fitting Time,633.236717
1,Generation Time,6032.498636
2,Post-Process Time,0.170545


In [32]:
data = pd.read_csv("travel insurance.csv")
modify_df = df[data.columns]

In [33]:
numerical_cols = modify_df.select_dtypes(include=['number']).columns
modify_df[numerical_cols] = modify_df[numerical_cols].round(0).astype(int)


In [34]:
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import evaluate_quality
from sdv.single_table import GaussianCopulaSynthesizer

In [35]:
metadata1 = SingleTableMetadata()
metadata1.detect_from_dataframe(data = data)

baseline_quality_report = evaluate_quality(
    data,  #real data
    modify_df, #synthetic data(perfect synthesizer generated(holdout))
    metadata1
)

Creating report: 100%|██████████| 4/4 [00:00<00:00,  4.19it/s]



Overall Quality Score: 93.64%

Properties:
Column Shapes: 98.16%
Column Pair Trends: 89.13%


## SDV

In [37]:
import time

data = pd.read_csv("travel insurance.csv")
metadata1 = SingleTableMetadata()

metadata1.detect_from_dataframe(data = data)
synthesier = GaussianCopulaSynthesizer(metadata1)
start_time = time.time()
synthesier.fit(data)
end_time = time.time()
fitting_time = end_time - start_time

start_time = time.time()
synthetic_data = synthesier.sample(num_rows = 63326)  #keep the number of rows equal to the data hold
end_time = time.time()
gen_time = end_time - start_time
synthetic_data

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,RAB,Airlines,Offline,Ticket Protector,No,50,INDONESIA,21.97,10.15,NaN,37
1,CWT,Travel Agency,Offline,Comprehensive Plan,No,13,AUSTRALIA,47.62,74.05,NaN,50
2,EPX,Airlines,Offline,Value Plan,No,42,VIET NAM,25.99,58.88,NaN,80
3,KML,Travel Agency,Offline,1 way Comprehensive Plan,No,1,UNITED STATES,32.44,7.18,F,31
4,KML,Travel Agency,Offline,Silver Plan,No,11,VIET NAM,37.52,23.61,NaN,50
...,...,...,...,...,...,...,...,...,...,...,...
63321,C2B,Airlines,Offline,Value Plan,No,-2,CZECH REPUBLIC,-9.73,18.56,NaN,26
63322,JZI,Airlines,Offline,Value Plan,No,21,"KOREA, DEMOCRATIC PEOPLE'S REPUBLIC OF",51.97,14.41,NaN,43
63323,C2B,Travel Agency,Offline,Cancellation Plan,No,11,MALAYSIA,-2.55,8.44,NaN,28
63324,CWT,Travel Agency,Offline,Bronze Plan,No,49,COSTA RICA,-24.23,11.02,NaN,48


In [38]:
# If fitting_time and y_data_time_used_gen are single values and you want them as separate rows
run_time_dict = {'process': ['fitting time', 'generation time'],
                 'time': [fitting_time, gen_time]}

run_time_df = pd.DataFrame(run_time_dict)

run_time_df

,process,time
0,fitting time,24.867424
1,generation time,2.384806


In [39]:
data = pd.read_csv("travel insurance.csv")
metadata1 = SingleTableMetadata()
metadata1.detect_from_dataframe(data = data)

baseline_quality_report = evaluate_quality(
    data,  #real data
    synthetic_data, #synthetic data(perfect synthesizer generated(holdout))
    metadata1
)

Creating report: 100%|██████████| 4/4 [00:00<00:00,  4.35it/s]



Overall Quality Score: 60.28%

Properties:
Column Shapes: 68.07%
Column Pair Trends: 52.49%


## GCSM

In [13]:
import GCSKM as G

In [14]:
import pandas as pd
data = pd.read_csv('travel insurance.csv')

In [15]:
import GCSKM as G
G = G.GaussianCopulaKmeansSynthesizer(data)
G._identify_columns()
G.convert_datetime_to_numerical()
G.handle_missing_values()
G.assign_intervals()
G.preprocess_data()
G.get_distribution()
G.calculate_cdfs()
G.standard_gaussian_all()
G.get_affinity()
G.get_laplacian()
G.optimal_clusters_dynamic()
G.spectral_embedding()
G.get_Kmeans()

G.generate_synthetic_data(63326)
df = G.post_process()

100%|██████████| 3/3 [00:04<00:00,  1.45s/it]


In [17]:
G.get_execution_times_df()


,Process,Time (seconds)
0,Fitting Time,3726.826745
1,Generation Time,169.243726
2,Post-Process Time,0.353211


In [19]:
df

,Duration,Gender,Product Name,Agency,Age,Commision (in value),Claim,Net Sales,Agency Type,Distribution Channel,Destination
0,0.007569,Nan,Rental Vehicle Excess Insurance,EPX,29.077672,17.697652,No,12.459106,Travel Agency,Online,AUSTRALIA
1,0.029305,F,Cancellation Plan,EPX,49.974903,14.470659,No,64.082767,Travel Agency,Online,UNITED KINGDOM
2,0.012396,Nan,Value Plan,EPX,50.228853,35.334311,No,52.851446,Travel Agency,Offline,CHINA
3,0.021811,Nan,2 way Comprehensive Plan,EPX,44.321367,9.176042,No,-13.956205,Travel Agency,Online,UNITED ARAB EMIRATES
4,0.018235,Nan,Rental Vehicle Excess Insurance,EPX,45.380070,7.024539,No,22.350428,Travel Agency,Online,HONG KONG
...,...,...,...,...,...,...,...,...,...,...,...
63321,0.006272,Nan,Cancellation Plan,EPX,23.697307,0.839274,No,-55.925581,Travel Agency,Online,INDONESIA
63322,0.023204,Nan,Basic Plan,EPX,42.576679,28.935422,No,-0.523133,Travel Agency,Offline,"TAIWAN, PROVINCE OF CHINA"
63323,0.012198,Nan,2 way Comprehensive Plan,EPX,46.046277,6.471765,No,20.274115,Travel Agency,Online,THAILAND
63324,0.022135,Nan,Cancellation Plan,EPX,33.775808,3.836523,No,28.340043,Travel Agency,Online,INDONESIA


In [22]:
import numpy as np
data = pd.read_csv("travel insurance.csv")
modify_df = df[data.columns]
numerical_cols = modify_df.select_dtypes(include=['number']).columns
modify_df[numerical_cols] = modify_df[numerical_cols].round(0).astype(int)

from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import evaluate_quality
from sdv.single_table import GaussianCopulaSynthesizer


metadata1 = SingleTableMetadata()
metadata1.detect_from_dataframe(data = data)

baseline_quality_report = evaluate_quality(
    data,  #real data
    modify_df, #synthetic data(perfect synthesizer generated(holdout))
    metadata1
)

Creating report: 100%|██████████| 4/4 [00:00<00:00,  4.68it/s]



Overall Quality Score: 76.98%

Properties:
Column Shapes: 74.52%
Column Pair Trends: 79.44%


## GMM

In [40]:
import numpy as np
import pandas as pd
from scipy.stats import kstest, norm, uniform, beta, expon, truncnorm
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import warnings
from joblib import Parallel, delayed
import scipy.stats as stats
from tqdm import tqdm
from scipy.optimize import brentq
import random
import time
warnings.filterwarnings(action='ignore')

class GaussianCopulaSynthesizer:

    """
    A class for synthesizing data using Gaussian Copula models.
    """
    def __init__(self, data):

        """
        Initialize the synthesizer with the provided file path.

        Parameters:
        - filepath (str): The file path to the input dataset.
        """

        # self.filepath = filepath
        self.data = data
        self.execution_times = {}
        self.num_column = []
        self.category_column = []
        self.datetime_column = []
        self.unique_identifier_columns = []
        self.special_numeric_series_columns = []
        self.special_column_prefixes = {}
        self.hyphenated_numeric_columns = []
        self.hyphenated_format = {}
        self.columns_with_missing_values = []
        self.binary_columns_for_missing = []
        self.category_intervals = {}
        self.gmms = {}
        self.column_cdfs = {}
        self.column_inverse_cdfs = {}
        self.processed_data = None
        self.synthetic_data = pd.DataFrame()
        self.digit_counts = {}  # Store digit counts for each column
        self.column_min = {}  # Store minimum values for each column
        self.column_max = {}

    def _start_timer(self):
        """
        Start the timer for a process.
        """
        self._timer_start = time.time()

    def _stop_timer(self, process_name):
        """
        Stop the timer and store the elapsed time for the process.

        Parameters:
        - process_name (str): The name of the process for which the timer was started.
        """
        elapsed_time = time.time() - self._timer_start
        self.execution_times[process_name] = elapsed_time


    def _identify_columns(self):
        """
        Identify and classify the columns of the dataset.

        Determines the type of each column (numerical, categorical, datetime, unique identifier, etc.)
        and classifies them into appropriate attributes of the class.
        """

        self._start_timer()
        self.unique_identifier_columns = []
        temp_special_columns = []
        temp_special_num_pattern = []

        data_info = self.data.dtypes.to_dict()
        for key, dtype in data_info.items():
            unique_values = self.data[key].nunique()
            total_values = len(self.data[key])

            # Check for uniqueness
            if unique_values == total_values:
                self.unique_identifier_columns.append(key)
                continue  # Skip further checks for this column
            
            if dtype == 'object' and pd.to_datetime(self.data[key], errors='coerce').notna().any():
                self.datetime_column.append(key)
            elif dtype == 'object':
                self.category_column.append(key)
            elif dtype in ['int64', 'float64']:
                self.num_column.append(key)

            if self.data[key].isnull().any():
                self.columns_with_missing_values.append(key)
        
        temp_column = self.unique_identifier_columns

        for column in temp_column:      
            try:
                if self.data[column].str.match(r'[A-Za-z]+\d+').all():
                    extracted, prefix_lengths, number_lengths = self.extract_number_part(self.data[column])
                    self.special_numeric_series_columns.append(column)
                    # Store extracted prefixes and their lengths
                    self.special_column_prefixes[column] = extracted[0].iloc[0]
                    self.digit_counts[column] = number_lengths.iloc[0]

                elif self.data[column].str.match(r'\d+-\d+-\d+').all():
                    # Handling hyphenated numeric columns
                    self.hyphenated_numeric_columns.append(column)
                    combined_numbers, digit_counts = self.extract_numbers(self.data[column])
                    self.hyphenated_format[column] = digit_counts.iloc[0].tolist()  # Store digit count format
            except AttributeError:
                self.unique_identifier_columns.remove(column)
                self.category_column.append(column)
        
        if self.hyphenated_numeric_columns == [] and self.special_numeric_series_columns == []:
            self.category_column.extend(self.unique_identifier_columns)
            self.unique_identifier_columns.clear()

        
               
        #self.unique_identifier_columns = [col for col in self.unique_identifier_columns if col not in temp_special_columns]
        self.special_numeric_series_columns.extend(temp_special_columns)

        #self.unique_identifier_columns = [col for col in self.unique_identifier_columns if col not in temp_special_num_pattern]
        self.hyphenated_numeric_columns.extend(temp_special_num_pattern)

        self._stop_timer("_identify_columns")
    
    def extract_number_part(self, series):
        """
        Extract alphanumeric prefix and numeric part from a series.

        Parameters:
        - series (pd.Series): A pandas Series from which to extract the alphanumeric prefix and numeric part.

        Returns:
        - tuple: A tuple containing the extracted prefix, prefix lengths, and number lengths.
        """
        
        # Extract both alphanumeric prefix and numeric part
        regex_pattern = r'([A-Za-z]+)(\d+)'
        extracted = series.str.extract(regex_pattern)
        # Calculate the length of each part (prefix and number)
        prefix_lengths = extracted[0].apply(lambda x: len(x) if pd.notnull(x) else 0)
        number_lengths = extracted[1].apply(lambda x: len(x) if pd.notnull(x) else 0)
        
        return extracted, prefix_lengths, number_lengths
    
    def extract_numbers(self, series):
        
        regex_pattern = r'(\d+)-(\d+)-(\d+)'
        extracted = series.str.extract(regex_pattern)

        # Calculate the number of digits in each part
        digit_counts = extracted.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)

        # Combine the numbers into a single string
        combined_numbers = extracted.apply(lambda row: '-'.join(row.dropna()), axis=1)

        

        return combined_numbers.astype(str), digit_counts

    
    def generate_random_hyphenated_number(self, format_pattern):
        """
        Generate a random number based on the observed format pattern.
        format_pattern: List of integers representing the length of each numeric segment.
        """
        random_number_parts = [str(random.randint(0, 10**length - 1)).zfill(length) for length in format_pattern]
        return '-'.join(random_number_parts)
    
    def generate_random_number(self, length):
        """
        Generate a random number of a specified length.
        """
        return str(random.randint(0, 10**length - 1)).zfill(length)
    
    def convert_datetime_to_numerical(self):
        ref_dt = pd.Timestamp('1970-01-01')

        for column in self.datetime_column:
            # Convert to datetime, coerce errors to NaT (missing values)
            self.data[column] = pd.to_datetime(self.data[column], errors='coerce')

            # Convert datetime to numerical value (e.g., days since reference date)
            self.data[column] = (self.data[column] - ref_dt) / np.timedelta64(1, 'D')

            # Reclassify as a numerical column
            self.num_column.append(column)
            if column in self.category_column:
                self.category_column.remove(column)
    
    def transform_to_original_format(self, column, synthetic_series):
        format_pattern = self.hyphenated_format[column]
        formatted_series = synthetic_series.apply(lambda x: '-'.join(part.zfill(length) for part, length in zip(x.split('-'), format_pattern)))
        return formatted_series

    def handle_missing_values(self):
        self._start_timer()
        for column in self.data.columns:
            if self.data[column].isnull().any():
                # Create a binary column to mark missing values
                binary_column_name = column + '_missing'
                self.data[binary_column_name] = self.data[column].isnull().astype(int)
                self.binary_columns_for_missing.append(binary_column_name)
                # Fill missing values in the original column
                self.data[column] = self.data[column].fillna(method='ffill').fillna(method='bfill')
        self._stop_timer("handle_missing_value")
    
    def convert_special_numerical(self):
        for column in self.special_numeric_series_columns:
            numerical_part = self.extract_number_part(self.data[column])
            self.data[column] = numerical_part



    def assign_intervals(self):
        self._start_timer()
        for column_name in (set(self.category_column)) - set(self.unique_identifier_columns) - set(self.binary_columns_for_missing):
            column_data = self.data[column_name]
            freq = column_data.value_counts(normalize=True)
            intervals = freq.cumsum()
            category_intervals = {}
            a = 0
            for category, cum_freq in intervals.items():
                b = cum_freq
                category_intervals[category] = (a, b)
                a = b
            self.category_intervals[column_name] = category_intervals
        self._stop_timer("assign_intervals")


    def sample_from_category(self, category_value, column_name):
        try:
            a, b = self.category_intervals[column_name][category_value]
            mean = (a + b) / 2
            sd = (b - a) / 6
            dist = truncnorm((0 - mean) / sd, (1 - mean) / sd, loc=mean, scale=sd)
            return dist.rvs()
        except KeyError:
            # Debugging information
            print(f"KeyError encountered in sample_from_category:")
            print(f"Column: {column_name}, Category Value: {category_value}")
            print(f"Available Categories in '{column_name}': {self.category_intervals[column_name]}")
            raise


    def preprocess_data(self):
        # self._identify_columns()
        # self.convert_datetime_to_numerical()
        # self.handle_missing_values()
        # self.assign_intervals()

        # Process only columns that are not unique identifiers
        self._start_timer()
        for column_name in tqdm(set(self.category_column) - set(self.unique_identifier_columns) - set(self.binary_columns_for_missing)):
            self.data[column_name] = self.data[column_name].apply(lambda x: self.sample_from_category(x, column_name))
        self._stop_timer("preprocess_data")


    
    def _find_best_components(self, data):
        bic = []
        n_components_range = range(1, 20)
        for n_components in n_components_range:
            gmm = GaussianMixture(n_components=n_components)
            gmm.fit(data)
            bic.append(gmm.bic(data))
        return n_components_range[np.argmin(bic)]

    # def _fit_gmm_for_column(self, column):
    #     best_components = self._find_best_components(self.data[column])
    #     gmm = GaussianMixture(n_components=best_components)
    #     gmm.fit(self.data[column].values.reshape(-1, 1))
    #     return column, gmm

    def fit_distributions(self):
        self._start_timer()
        # Include normal numeric and categorical columns (excluding unique identifiers and binary columns for missing values)
        self.relevant_columns = (set(self.num_column) | set(self.category_column)) - set(self.unique_identifier_columns) - set(self.binary_columns_for_missing)

        data_for_gmm = self.data[self.relevant_columns]
        best_components = self._find_best_components(data_for_gmm)

        gmm = GaussianMixture(n_components = best_components)

        gmm.fit(data_for_gmm)

        self.gmm = gmm

        for column in self.relevant_columns:
            self.column_min[column] = self.data[column].min()
            self.column_max[column] = self.data[column].max()

        self._stop_timer('fit_distribution')
    
    def get_gmm(self, column_name):
        return self.gmms.get(column_name, None) #find the gmm for that column, if not found, return None

    def generate_synthetic_data(self, num_rows):

        self._start_timer()
        # Generate synthetic data for regular columns using GMMs
        self.relevant_columns = list((set(self.data.columns) - set(self.binary_columns_for_missing)) - set(self.unique_identifier_columns))

        samples = self.gmm.sample(n_samples=num_rows)[0]  # Get samples from GMM
        synthetic_df = pd.DataFrame(samples, columns=self.relevant_columns)

      
        
        for column in self.relevant_columns:
          min_val, max_val = self.column_min.get(column, None), self.column_max.get(column, None)
          if min_val is not None and max_val is not None:
              synthetic_df[column] = synthetic_df[column].clip(lower=min_val, upper=max_val)
      
        self.synthetic_data = synthetic_df.fillna(synthetic_df.mean())

        for column in self.special_numeric_series_columns:
            prefix = self.special_column_prefixes[column]
            number_length = self.digit_counts[column]
            # Generate synthetic data based on extracted structure
            self.synthetic_data[column] = [
                prefix + self.generate_random_number(number_length)
                for _ in range(num_rows)
            ]

        for column in self.hyphenated_numeric_columns:
            format_pattern = self.hyphenated_format.get(column, [])
            self.synthetic_data[column] = [
                self.generate_random_hyphenated_number(format_pattern) for _ in range(num_rows)
            ]
        self._stop_timer("generate_syn_data")
        return self.synthetic_data
        
        

    def numerical_to_category(self, num_value, column_name):
        """ Convert a numerical value back to its corresponding category. """
        for category, (a, b) in self.category_intervals[column_name].items():
            if a <= num_value < b:
                return category
            elif num_value > 1 and round(b) == 1:
                return category
            elif num_value < 0 and round(a) == 0:
                return category
        return None  # Return None or some default value if no category matches
    
    def numerical_to_datetime(self, num_value):
        """
        Convert a numerical value back to its corresponding datetime.
        """
        return pd.Timestamp("1970-01-01") + pd.to_timedelta(num_value, unit='s')

    def post_process(self):
        """
        Convert all numerical columns back to their original categorical or datetime form.

        This method converts columns that were previously transformed into a numerical format
        back to their original categorical or datetime format.

        Returns:
        - pd.DataFrame: A DataFrame with categorical columns converted back to their original categories
                        and datetime columns converted back to their original datetime format.
        """
        self._start_timer()
        for column_name in set(self.category_column) - set(self.unique_identifier_columns) - set(self.binary_columns_for_missing):
            self.synthetic_data[column_name] = self.synthetic_data[column_name].apply(lambda x: self.numerical_to_category(x, column_name))

        for column_name in self.datetime_column:
            self.synthetic_data[column_name] = self.synthetic_data[column_name].round().astype(int)
        # Ensure the column is numeric and represents days since the epoch
            self.synthetic_data[column_name] = pd.to_timedelta(self.synthetic_data[column_name], unit='d') + pd.Timestamp("1970-01-01")

        for column in self.columns_with_missing_values:
            original_column = column.replace('_missing', '')
            self.synthetic_data.loc[self.data[column+'_missing'] == 1, original_column] = np.nan
        
        self._stop_timer("post_process")
        return self.synthetic_data
    
    def get_execution_times_df(self):
        """
        Convert the execution times dictionary to a DataFrame.

        Returns:
        - pd.DataFrame: A DataFrame with process names and their corresponding execution times.
        """
        return pd.DataFrame(list(self.execution_times.items()), columns=['Process', 'Time (seconds)'])
    
    def get_execution_times_df(self):
        """
        Convert the execution times dictionary to a DataFrame and calculate
        fitting time, generation time, and post-process time.

        Returns:
        - pd.DataFrame: A DataFrame with the detailed process names and their corresponding execution times.
        - pd.DataFrame: A DataFrame with summarized fitting time, generation time, and post-process time.
        """
        # Create the DataFrame from the execution times dictionary
        execution_times_df = pd.DataFrame(list(self.execution_times.items()), columns=['Process', 'Time (seconds)'])

        # Calculate the 'Fitting Time' as the sum of times for indices 0 to 6
        fitting_time = execution_times_df.iloc[0:5]['Time (seconds)'].sum() if len(execution_times_df) > 5 else 0

        # 'Generation Time' is the time for index 7
        generation_time = execution_times_df.iloc[5]['Time (seconds)'] if len(execution_times_df) > 5 else 0

        # 'Post-Process Time' is the time for index 8
        post_process_time = execution_times_df.iloc[6]['Time (seconds)'] if len(execution_times_df) > 6 else 0

        # Create a summary DataFrame
        summary_df = pd.DataFrame({
            'Process': ['Fitting Time', 'Generation Time', 'Post-Process Time'],
            'Time (seconds)': [fitting_time, generation_time, post_process_time]
        })

        # Return both the detailed and summary DataFrames
        return summary_df

In [41]:
import pandas as pd
data = pd.read_csv("travel insurance.csv")

data

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,81
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,71
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65,AUSTRALIA,-49.5,29.70,NaN,32
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60,AUSTRALIA,-39.6,23.76,NaN,32
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79,ITALY,-19.8,11.88,NaN,41
...,...,...,...,...,...,...,...,...,...,...,...
63321,JZI,Airlines,Online,Basic Plan,No,111,JAPAN,35.0,12.25,M,31
63322,JZI,Airlines,Online,Basic Plan,No,58,CHINA,40.0,14.00,F,40
63323,JZI,Airlines,Online,Basic Plan,No,2,MALAYSIA,18.0,6.30,M,57
63324,JZI,Airlines,Online,Basic Plan,No,3,VIET NAM,18.0,6.30,M,63


In [42]:
G = GaussianCopulaSynthesizer(data)
G._identify_columns()
G.convert_datetime_to_numerical()
G.handle_missing_values()
G.assign_intervals()
G.preprocess_data()
G.fit_distributions()
G.generate_synthetic_data(63326)
G.post_process()
G.get_execution_times_df()

100%|██████████| 7/7 [02:46<00:00, 23.78s/it]


,Process,Time (seconds)
0,Fitting Time,275.856285
1,Generation Time,0.044010
2,Post-Process Time,0.175994


In [43]:
data = pd.read_csv("travel insurance.csv")

df = G.synthetic_data
modify_df = df[data.columns]
modify_df

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,JWT,Airlines,Online,Value Plan,No,19.496674,INDIA,88.163399,35.262919,M,117.999751
1,JWT,Airlines,Online,Value Plan,No,89.917760,INDIA,78.496079,31.398435,F,118.000000
2,JWT,Airlines,Online,Value Plan,No,6.927942,INDIA,82.022469,32.807723,NaN,117.999033
3,JWT,Airlines,Online,Value Plan,No,67.074496,INDIA,62.630943,25.050137,NaN,118.000000
4,JWT,Airlines,Online,Value Plan,No,28.609346,INDIA,85.009341,34.002398,NaN,118.000000
...,...,...,...,...,...,...,...,...,...,...,...
63321,EPX,Travel Agency,Online,1 way Comprehensive Plan,No,60.231495,HONG KONG,19.999594,0.000000,F,29.893547
63322,EPX,Travel Agency,Online,1 way Comprehensive Plan,No,-2.000000,SINGAPORE,20.000430,0.000000,M,39.110468
63323,EPX,Travel Agency,Online,1 way Comprehensive Plan,No,1.683706,INDONESIA,19.999512,0.000231,F,28.177947
63324,EPX,Travel Agency,Online,1 way Comprehensive Plan,No,21.158026,AUSTRALIA,19.998725,0.000000,M,31.168269


In [44]:
numerical_cols = modify_df.select_dtypes(include=[np.number]).columns.tolist()


modify_df[numerical_cols] = modify_df[numerical_cols].astype(int)

In [45]:
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import evaluate_quality
from sdv.single_table import GaussianCopulaSynthesizer

In [46]:
data = pd.read_csv("travel insurance.csv")
metadata1 = SingleTableMetadata()
metadata1.detect_from_dataframe(data = data)

baseline_quality_report = evaluate_quality(
    data,  #real data
    modify_df, #synthetic data(perfect synthesizer generated(holdout))
    metadata1
)

Creating report: 100%|██████████| 4/4 [00:00<00:00,  4.79it/s]



Overall Quality Score: 91.34%

Properties:
Column Shapes: 92.94%
Column Pair Trends: 89.75%


## GCKM_s

In [1]:
import pandas as pd
import numpy as np

In [2]:
import GCKM_s as G

data = pd.read_csv("travel insurance.csv")

G = G.GaussianCopulaKmeansSynthesizer(data)
G._identify_columns()
G.convert_datetime_to_numerical()
G.handle_missing_values()
G.assign_intervals()
G.preprocess_data()
G.get_distribution()
G.calculate_cdfs()
G.standard_gaussian_all()
G.optimal_clusters_dynamic()
G.get_Kmeans()
# G.generate_data()
G.generate_synthetic_data(63326)
G.post_process()
G.get_execution_times_df()

100%|██████████| 2/2 [01:21<00:00, 40.54s/it]


,Process,Time (seconds)
0,Fitting Time,635.500384
1,Generation Time,81.242932
2,Post-Process Time,0.157539


In [3]:
synthetic_data = G.synthetic_data

numerical_cols = synthetic_data.select_dtypes(include=[np.number]).columns.tolist()


synthetic_data[numerical_cols] = synthetic_data[numerical_cols].astype(int)

synthetic_data

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,ART,Travel Agency,Online,Cancellation Plan,No,-2,LIBYAN ARAB JAMAHIRIYA,98,52,F,56
1,EPX,Travel Agency,Online,Cancellation Plan,No,-2,PHILIPPINES,33,26,M,39
2,EPX,Travel Agency,Online,Rental Vehicle Excess Insurance,No,22,"KOREA, REPUBLIC OF",0,22,NaN,49
3,CWT,Travel Agency,Online,Gold Plan,No,-2,SINGAPORE,38,26,NaN,35
4,C2B,Travel Agency,Online,Cancellation Plan,No,86,CHINA,56,26,NaN,16
...,...,...,...,...,...,...,...,...,...,...,...
63321,CWT,Airlines,Online,Cancellation Plan,No,72,AUSTRALIA,75,23,M,41
63322,EPX,Travel Agency,Online,Rental Vehicle Excess Insurance,No,34,JAPAN,18,13,M,41
63323,C2B,Travel Agency,Online,Rental Vehicle Excess Insurance,No,-2,AUSTRALIA,66,20,M,44
63324,C2B,Travel Agency,Online,Basic Plan,No,23,UNITED STATES,41,31,F,41


In [4]:
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import evaluate_quality
from sdv.single_table import GaussianCopulaSynthesizer

data = pd.read_csv("travel insurance.csv")
metadata1 = SingleTableMetadata()
metadata1.detect_from_dataframe(data = data)

baseline_quality_report = evaluate_quality(
    data,  #real data
    synthetic_data, #synthetic data(perfect synthesizer generated(holdout))
    metadata1
)

Creating report: 100%|██████████| 4/4 [00:00<00:00,  4.93it/s]



Overall Quality Score: 82.72%

Properties:
Column Shapes: 81.82%
Column Pair Trends: 83.62%


## GCKM_e

In [1]:
import GCKM_e as G
import pandas as pd

data = pd.read_csv("travel insurance.csv")
G = G.GaussianCopulaKmeansSynthesizer(data)
G._identify_columns()
G.convert_datetime_to_numerical()
G.handle_missing_values()
G.assign_intervals()
G.preprocess_data()
G.get_distribution()
G.calculate_cdfs()
G.standard_gaussian_all()
G.optimal_clusters_dynamic()
G.get_Kmeans()
# G.generate_data()
G.generate_synthetic_data(63326)
G.post_process()
G.get_execution_times_df()

100%|██████████| 2/2 [01:21<00:00, 40.70s/it]


,Process,Time (seconds)
0,Fitting Time,210.750804
1,Generation Time,81.537179
2,Post-Process Time,0.162045


In [3]:
import numpy as np
synthetic_data = G.synthetic_data

numerical_cols = synthetic_data.select_dtypes(include=[np.number]).columns.tolist()


synthetic_data[numerical_cols] = synthetic_data[numerical_cols].astype(int)

synthetic_data

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,EPX,Travel Agency,Online,1 way Comprehensive Plan,No,53,THAILAND,-110,21,F,-4
1,C2B,Travel Agency,Online,2 way Comprehensive Plan,No,83,PHILIPPINES,18,18,M,37
2,EPX,Travel Agency,Online,Rental Vehicle Excess Insurance,No,74,MALAYSIA,-5,13,NaN,26
3,EPX,Travel Agency,Online,Value Plan,No,-2,SINGAPORE,125,8,NaN,40
4,JZI,Travel Agency,Online,Cancellation Plan,No,120,SINGAPORE,97,21,NaN,62
...,...,...,...,...,...,...,...,...,...,...,...
63321,C2B,Travel Agency,Online,Cancellation Plan,No,96,ISRAEL,35,21,M,44
63322,EPX,Travel Agency,Online,Value Plan,No,115,CANADA,41,14,F,40
63323,CWT,Airlines,Online,Basic Plan,No,-2,SINGAPORE,45,23,M,38
63324,JZI,Travel Agency,Online,Cancellation Plan,No,72,INDIA,27,21,M,38


In [4]:
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import evaluate_quality
from sdv.single_table import GaussianCopulaSynthesizer

data = pd.read_csv("travel insurance.csv")
# data = data.iloc[:1000,:]
metadata1 = SingleTableMetadata()
metadata1.detect_from_dataframe(data = data)

baseline_quality_report = evaluate_quality(
    data,  #real data
    synthetic_data, #synthetic data(perfect synthesizer generated(holdout))
    metadata1
)

Creating report: 100%|██████████| 4/4 [00:00<00:00,  4.98it/s]



Overall Quality Score: 83.59%

Properties:
Column Shapes: 83.34%
Column Pair Trends: 83.84%
